In [50]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [3]:
r = requests.get('https://www.yelp.com/user_details_reviews_self?userid=37cpUoM8hlkSQfReIEBd-Q&rec_pagestart=0')

In [4]:
r.request.headers

{'Connection': 'keep-alive', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'User-Agent': 'python-requests/2.12.4'}

In [5]:
r.content

'<!DOCTYPE HTML>\n\n<!--[if lt IE 7 ]> <html xmlns:fb="http://www.facebook.com/2008/fbml" class="ie6 ie ltie9 ltie8 no-js" lang="en"> <![endif]-->\n<!--[if IE 7 ]>    <html xmlns:fb="http://www.facebook.com/2008/fbml" class="ie7 ie ltie9 ltie8 no-js" lang="en"> <![endif]-->\n<!--[if IE 8 ]>    <html xmlns:fb="http://www.facebook.com/2008/fbml" class="ie8 ie ltie9 no-js" lang="en"> <![endif]-->\n<!--[if IE 9 ]>    <html xmlns:fb="http://www.facebook.com/2008/fbml" class="ie9 ie no-js" lang="en"> <![endif]-->\n<!--[if (gt IE 9)|!(IE)]><!--> <html xmlns:fb="http://www.facebook.com/2008/fbml" class="no-js" lang="en"> <!--<![endif]-->\n    <head>\n\n\n    <script>            window.yPageStart = new Date().getTime();\n</script>\n\n    <script>            var initialVisibilityState = document.webkitVisibilityState;\n\n\n            (function(H){H.className=H.className.replace(/\\bno-js\\b/,\'js\');})(document.documentElement);\n</script>\n\n            <script>            (function(i,s,o,g,r,

In [6]:
soup = BeautifulSoup(r.content, "lxml")

In [7]:
print(soup.prettify())

<!DOCTYPE HTML>
<!--[if lt IE 7 ]> <html xmlns:fb="http://www.facebook.com/2008/fbml" class="ie6 ie ltie9 ltie8 no-js" lang="en"> <![endif]-->
<!--[if IE 7 ]>    <html xmlns:fb="http://www.facebook.com/2008/fbml" class="ie7 ie ltie9 ltie8 no-js" lang="en"> <![endif]-->
<!--[if IE 8 ]>    <html xmlns:fb="http://www.facebook.com/2008/fbml" class="ie8 ie ltie9 no-js" lang="en"> <![endif]-->
<!--[if IE 9 ]>    <html xmlns:fb="http://www.facebook.com/2008/fbml" class="ie9 ie no-js" lang="en"> <![endif]-->
<!--[if (gt IE 9)|!(IE)]><!-->
<html class="no-js" lang="en" xmlns:fb="http://www.facebook.com/2008/fbml">
 <!--<![endif]-->
 <head>
  <script>
   window.yPageStart = new Date().getTime();
  </script>
  <script>
   var initialVisibilityState = document.webkitVisibilityState;


            (function(H){H.className=H.className.replace(/\bno-js\b/,'js');})(document.documentElement);
  </script>
  <script>
   (function(i,s,o,g,r,a,m){i['GoogleAnalyticsObject']=r;i[r]=i[r]||function(){(i[r].q=i

In [38]:
reviewfortype= soup.find(class_="review")
typnone= type(reviewfortype.find('span',{'class':"business-attribute price-range"}))

In [40]:
typnone

NoneType

In [51]:
#functions to extract info for each restaurant will come after "for d in soup.findAll(class_="review"):" 
#which iterates through restaurants

## name of restaurant 
def extract_name(d): 
    try: 
        name = (d.find(class_="biz-name js-analytics-click").text.strip()) 
    except: 
        name = '' 
    return name
 
## URL for restaurant
def extract_RURL(d): 
    try: 
        RURL = 'https://www.yelp.com'+ d.find(class_="biz-name js-analytics-click")['href']
    except: 
        RURL = '' 
    return RURL
    
def extract_genre(d): 
    try: 
        typperreview=[]
        for typ in d.find(class_="category-str-list").findAll('a'):
            typperreview.append(str(typ.get_text())) 
        genre = typperreview
    except: 
        genre = '' 
    return genre

def extract_address(d): 
    try: 
        address = [str(x.strip()) for x in d.find('address').find_all(text=True)]
    except: 
        address = '' 
    return address

def extract_rating(d): 
    try: 
        rating = str(d.find(class_="i-stars")['title'])[:3]
    except: 
        rating = '' 
    return rating

def extract_date(d): 
    try: 
        date = d.find(class_="rating-qualifier").get_text().strip()
    except: 
        date = '' 
    return date

def extract_price(d): 
    try: 
        if type(d.find('span',{'class':"business-attribute price-range"}))==typnone:
            price = ''
        else:
            price = d.find('span',{'class':"business-attribute price-range"}).get_text()
    except: 
        price = '' 
    return price

In [54]:
#define empyt dict for data
rest_dict = [] 

#change if want more results
MAX_RESULTS_PER_REVIEWER = 1330

#input user id, can be found in URL from Yelp profile
userid = '37cpUoM8hlkSQfReIEBd-Q'

#iterate through pages... only works with pages that have 10 resultes
for start in range(0, MAX_RESULTS_PER_REVIEWER, 10):
    URL = 'https://www.yelp.com/user_details_reviews_self?userid=' + userid + '&rec_pagestart=' + str(start)
    r=requests.get(URL)
    soup = BeautifulSoup(r.content, "lxml")

    #iterate through reviews on that page
    for i in soup.findAll(class_="review"):

        name = extract_name(i)
        date = extract_date(i)
        RURL = extract_RURL(i)
        genre = extract_genre(i)
        address = extract_address(i)
        rating = extract_rating(i)
        price = extract_price(i)
        
        #add restaurant review to dict
        rest_dict.append([name, date, RURL, genre, address, rating, price])

#make dictionary into dataframe
rest_df = pd.DataFrame(rest_dict, columns=['Rest_Name', 'Date', 'RURL', 'Genre', 'Address', 'Rating', 'Dolla_Signs'])       

#export data to csv
## CHANGE NAME
rest_df.to_csv('scrapeMikeChu'+str(MAX_RESULTS_PER_REVIEWER)+'.csv', encoding='utf-8')

In [57]:
mikereviews = pd.read_csv('/Users/samfalk/Fun Projects/scrapeMikeChu1330.csv')

In [58]:
mikereviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1330 entries, 0 to 1329
Data columns (total 8 columns):
Unnamed: 0     1330 non-null int64
Rest_Name      1330 non-null object
Date           1330 non-null object
RURL           1330 non-null object
Genre          1328 non-null object
Address        1330 non-null object
Rating         1330 non-null float64
Dolla_Signs    1223 non-null object
dtypes: float64(1), int64(1), object(6)
memory usage: 83.2+ KB


In [46]:
#testing output of bsoup
for d in soup.findAll(class_="review"):
    print d.find(class_="biz-name js-analytics-click").text

    print 'https://www.yelp.com'+ d.find(class_="biz-name js-analytics-click")['href']
    typperreview=[]
    for typ in d.find(class_="category-str-list").findAll('a'):
        typperreview.append(str(typ.get_text()))
    print typperreview
    print [str(x.strip()) for x in d.find('address').find_all(text=True)]
    print str(d.find(class_="i-stars")['title'])[:3]
    print d.find(class_="rating-qualifier").get_text().strip() 
    if type(d.find('span',{'class':"business-attribute price-range"}))==typnone:
        print 'None'
    else:
        print d.find('span',{'class':"business-attribute price-range"}).get_text()
    print

The Meatball Shop
https://www.yelp.com/biz/the-meatball-shop-new-york-11
['Italian', 'American (Traditional)']
['798 9th Ave', 'New York, NY 10019']
5.0
3/1/2017
None

ABCV
https://www.yelp.com/biz/abcv-new-york-2
['American (Traditional)', 'Bars', 'Breakfast & Brunch']
['38 East 19th St', 'New York, NY 10003']
5.0
2/27/2017
None

L’estudio
https://www.yelp.com/biz/l-estudio-new-york
['Mediterranean', 'Breakfast & Brunch', 'Sandwiches']
['61 Hester St', 'New York, NY 10002']
5.0
2/26/2017
None

Whitman’s
https://www.yelp.com/biz/whitmans-new-york-44
['Burgers', 'Desserts', 'American (New)']
['331 10th Ave', 'New York, NY 10001']
5.0
2/25/2017
None

Trapizzino
https://www.yelp.com/biz/trapizzino-new-york
['Italian']
['144 Orchard St', 'New York, NY 10002']
4.0
2/25/2017
$

Ikinari Steak East Village
https://www.yelp.com/biz/ikinari-steak-east-village-new-york
['Steakhouses', 'Japanese', 'Beer, Wine & Spirits']
['90 E 10th St', 'New York, NY 10003']
5.0
2/22/2017
$$

Atlantic Social
http

In [ ]:
##FOR REFERENCE FROM ANOTHER PROJECT 
# import requests import bs4 import pandas as pd from bs4 import BeautifulSoup import datetime import urllib

# def extract_location_fromresultRow(result): try: location = (result.find(class='location').text.strip()) except: location = '' return location
# def extract_company_fromresultRow(result): try: company = (result.find(class='company').text.strip()) except: company = '' return company
# def extract_jkid_fromresultRow(result): try: row = (result.find(class='jobtitle turnstileLink')) jkid = result['data-jk'] except: jkid = '' return jkid
# def extract_title_fromresultRow(result): try: title = (result.find(class='turnstileLink')) title_text = title.text except: title_text = '' return title_text
# def extract_salary_fromresultRow(result): try: salary = (result.find(class='snip').find('nobr').text) except: salary = '' salary_text = salary return salary_text
# def extract_reviews_fromresultRow(result): try: reviews = (result.find(class='slNoUnderline').text.strip().strip(' reviews').replace(',','')) except: reviews = '' return reviews
# def extract_stars_fromresultRow(result): try: stars = (result.find(class='rating')['style']).split(';background-position:')[1].split(':')[1].split('px')[0].strip() except: stars = '' return stars
# def extract_date_fromresultRow(result): try: date = (result.find(class='date').text.strip(' ago').strip()) except: date = '' return date


# for city in CITY_SET: job_dict = [] now = datetime.datetime.now() for start in range(0, MAX_RESULTS_PER_CITY, 10):
#     URL = "http://www.indeed.com/jobs?q="+urllib.quote(URL_SEARCH_TERM)+"&l="+urllib.quote(city)+"&start="+str(start)
#     r=requests.get(URL)
#     soup = BeautifulSoup(r.content, "lxml")

#     for i in soup.findAll("div", {"data-tn-component" : "organicJob"}):

#         location = extract_location_from_resultRow(i)
#         company = extract_company_from_resultRow(i)
#         jkid = extract_jkid_from_resultRow(i)
#         title = extract_title_from_resultRow(i)
#         salary = extract_salary_from_resultRow(i)
#         reviews = extract_reviews_from_resultRow(i)
#         stars = extract_stars_from_resultRow(i)
#         post_date = extract_date_from_resultRow(i)

#         job_dict.append([location, company, jkid,title, salary, stars, reviews, post_date, now])

#     job_df = pd.DataFrame(job_dict, columns=['location', 'company', 'jkid', 'title', 'salary', 'stars', 'reviews', 'post_date', 'pull_date'])       

# job_df.to_csv('scrape'+city+'_'+str(MAX_RESULTS_PER_CITY)+'.csv', encoding='utf-8')